In [ ]:
!pip install trl transformers accelerate git+https://github.com/huggingface/peft.git -Uqqq
!pip install datasets bitsandbytes einops wandb -Uqqq

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.5

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-base")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")
model.config.forced_decoder_ids = None


In [ ]:

# load dummy dataset and read audio files
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
sample = ds[0]["audio"]
input_features = processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").input_features

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
# generate token ids
predicted_ids = model.generate(input_features)
# decode token ids to text
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

# Tarteel

In [ ]:
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq, Wav2Vec2ForCTC, Wav2Vec2Tokenizer


In [ ]:
processor = AutoProcessor.from_pretrained("ZahrizhalAli/quran-tarteel-finetune-whisperbase")
model = AutoModelForSpeechSeq2Seq.from_pretrained("ZahrizhalAli/quran-tarteel-finetune-whisperbase")

In [ ]:
from transformers import pipeline

# Create an ASR pipeline using the specified model
pipe = pipeline("automatic-speech-recognition", model="ZahrizhalAli/quran-tarteel-finetune-whisperbase")




In [ ]:
# Define your audio input path (replace with the actual path to your audio file)
audio_file_path = "/content/001001.mp3"  # Replace with the correct path
sampling_rate = 16000



In [ ]:
# Perform ASR inference on the audio file
results = pipe(audio_file_path)



/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (1024) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
results['text']

'بِسْمِ اللَّهِ الرَّحْمَنِ الرَّحِيمِ'

In [ ]:
def lihat_kesalahan(sentence1, sentence2):
    highlighted = ""
    if sentence1 == sentence2:
      return "All Correct"
    for char1, char2 in zip(sentence1, sentence2):
        if char1 != char2:
            highlighted += f"[{char2}]"
        else:
            highlighted += char2

    return highlighted

# Example sentences to compare
correct_sentence = results['text']
wrong_sentence = "بِسْمِ اللَّهِهِ الرَّحْمَنِ الرَّحِييمِ"
# wrong_sentence = "بِسْمِ اللَّهِ الرَّحْمَنِ الرَّحِيمِ"

# Highlight the differences
highlighted_sentence = lihat_kesalahan(correct_sentence, wrong_sentence)

# Print the highlighted sentence
print("Correct Sentence:", correct_sentence)
print("Wrong Sentence:", highlighted_sentence)


Correct Sentence: بِسْمِ اللَّهِ الرَّحْمَنِ الرَّحِيمِ
Wrong Sentence: بِسْمِ اللَّهِ[ه][ِ][ ][ا][ل][ر][ّ][َ][ح][ْ][م][َ][ن][ِ][ ][ا][ل][ر][ّ][َ][ح][ِ][ي]


In [ ]:
# def compare_arabic_sentences(sentence1, sentence2):
#     differences = []

#     for idx, (char1, char2) in enumerate(zip(sentence1, sentence2)):
#         if char1 != char2:
#             differences.append((idx, char1, char2))

#     return differences

# # Example sentences to compare
# sentence1 = results['text']
# sentence2 = "بِسْمِ اللَّهِ الرَّحْمَنِ الرَّحِييمِ"

# # Compare the sentences
# differences = compare_arabic_sentences(sentence1, sentence2)

# # Print the differences
# if not differences:
#     print("The sentences are identical.")
# else:
#     print("Differences:")
#     for idx, char1, char2 in differences:
#         print(f"At position {idx}: '{char1}' in sentence 1, '{char2}' in sentence 2")


Differences:
At position 35: 'م' in sentence 1, 'ي' in sentence 2
At position 36: 'ِ' in sentence 1, 'م' in sentence 2
